In [9]:
# from airflow import DAG
# from airflow.operators.python import PythonOperator
# from airflow.models import Variable
# from airflow.utils import timezone
# from jinja2 import Environment, FileSystemLoader
# from datetime import timedelta
import smtplib
import psycopg2
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

def send_bill_reminders():
    # Load vars
    db_config = {
        'host': Variable.get("DB_HOST"),
        'port': Variable.get("DB_PORT"),
        'dbname': Variable.get("DB_NAME"),
        'user': Variable.get("DB_USER"),
        'password': Variable.get("DB_PASSWORD"),
    }

    smtp_host = Variable.get("SMTP_HOST")
    smtp_port = int(Variable.get("SMTP_PORT"))
    email_sender = Variable.get("EMAIL_SENDER")
    email_password = Variable.get("EMAIL_PASSWORD")

    # Connect to DB
    conn = psycopg2.connect(**db_config)
    cursor = conn.cursor()

    query = """
    SELECT u.full_name, u.email, rb.due_date, rb.amount
    FROM users u
    JOIN bookings b ON u.user_id = b.user_id
    JOIN recurring_bills rb ON b.booking_id = rb.booking_id
    WHERE rb.is_paid = FALSE
        AND rb.due_date = CURRENT_DATE + INTERVAL '10 days';
    """
    cursor.execute(query)
    rows = cursor.fetchall()
    print(f"Fetched {len(rows)} rows from the database.")

    if not rows:
        print("✅ No upcoming bills found.")
        return

    # Load HTML template
    template = env.get_template('email_template.html')

    for full_name, email, due_date, amount in rows:
        html_content = template.render(
            full_name=full_name,
            due_date=due_date.strftime('%Y-%m-%d'),
            amount=f"{amount:,.2f}"
        )

        msg = MIMEMultipart('alternative')
        msg['Subject'] = "⏰ Kosan Payment Reminder"
        msg['From'] = email_sender
        msg['To'] = email
        print(f"Preparing email for {full_name} <{email}>: Due {due_date.strftime('%Y-%m-%d')}, Amount {amount:,.2f}")
        msg.attach(MIMEText(html_content, 'html'))

        try:
            with smtplib.SMTP(smtp_host, smtp_port) as server:
                server.starttls()
                server.login(email_sender, email_password)
                server.sendmail(email_sender, email, msg.as_string())
            print(f"✅ Reminder sent to: {email}")
        except Exception as e:
            print(f"❌ Failed to send email to {email}: {e}")

    cursor.close()
    conn.close()

In [24]:
db_config = {
    'host': '34.50.70.80',
    'port': '5432',
    'dbname': 'hackathon-project',
    'user': 'hackathonProject',
    'password': 'hackathonProject!23',
}

smtp_host = 'smtp.gmail.com'
smtp_port = 587
email_sender = 'rhecopk@gmail.com'
email_password = 'cbdytrozkhuucdpq'

# Connect to DB
conn = psycopg2.connect(**db_config)
cursor = conn.cursor()

query = """
SELECT u.full_name, u.email, rb.due_date, rb.amount
FROM users u
JOIN bookings b ON u.user_id = b.user_id
JOIN recurring_bills rb ON b.booking_id = rb.booking_id
WHERE rb.is_paid = FALSE
  AND rb.due_date::date = CURRENT_DATE + INTERVAL '10 days';
"""
cursor.execute(query)
rows = cursor.fetchall()
print(f"Fetched {len(rows)} rows from the database.")

Fetched 0 rows from the database.


In [ ]:
cursor.execute("")  # or your actual timezone

In [23]:
cursor.execute("SELECT CURRENT_DATE, CURRENT_DATE + INTERVAL '10 days';")
print("DB Date:", cursor.fetchone())
rows

DB Date: (datetime.date(2025, 6, 5), datetime.datetime(2025, 6, 15, 0, 0))


[]